# Curso WRF-python

Gonzalo Zigarán

_gjzigaran@gmail.com_

#### Links:
* https://github.com/NCAR/wrf-python/
* https://github.com/NCAR/wrf_python_tutorial/

## Temas:
1. Introducción a las librerías (python, jupyter notebooks, numpy, pandas, xarray)
1. Cómo leer una salida de WRF 
1. wrf-python
1. Librerías para graficar (matplotlib, basemap)
1. Ejemplos

### 1. Introducción a las librerías

#### Python

- Lenguaje de programación relativamente nuevo y con gran crecimiento
- Es interpretado 
- Licencia de código abierto
- De alto nivel, lo que posibilita que su código sea muy legible.
- Gran cantidad de documentación y comunidad de usuarios


#### Jupyter Notebook

Es una aplicación web de codigo abierto que permite crear documentos que combinan código, visualizaciones y texto, además de ejecutar el código. Originalmente estaba pensado para el intérprete IPython pero ahora también permite trabajar con otros lenguajes como Julia, R, Octave.

#### Numpy

NumPy es un paquete para computación científica con Python. Le agrega un mayor soporte para el trabajo con vectores y matrices eficientemente, integrado con C y Fortran.

Los arreglos de numpy pueden ser creados con los tipos comunes de C:
- int8, int16, int32, int64 
- float16, float32, float64 [default]
- bool
- complex64, complex128
y pueden ser ordenados como en C (más rápido de acceder a derecha, por defecto) o como en Fortran (más rápido de acceder a izquierda)

##### Ejemplo de uso:


In [1]:
import numpy as np

# crea un un arreglo tri-dimensional de ceros
my_array = np.zeros((3,3,3), "float32")

# acceder a un elemento
first_element = my_array[0,0,0]

# editar un elemento
my_array[0,0,1] = 5

# acceder a una fila
first_row = my_array[0,0,:]

print my_array
print first_element
print first_row

[[[ 0.  5.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]]
0.0
[ 0.  5.  0.]



#### Pandas

Pandas es una librería que proporciona alto rendimiento para el análisis y estructura de datos. Es fácil e intuitivo de usar y conbina muchas funciones con NumPy. (Gran reemplazo del excel)

##### Ejemplo de uso:


In [2]:
import pandas as pd

d = {'one' : pd.Series([1., 2., 3.], index=['a', 'b', 'c']),
     'two' : pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd'])} 

df = pd.DataFrame(d)


df

,one,two
a,1.0,1.0
b,2.0,2.0
c,3.0,3.0
d,NaN,4.0


In [3]:
df2 = pd.DataFrame(columns=['one','two'])

df2.loc['a'] = [1,1]
df2.loc['b'] = [2,2]
df2.loc['c'] = [3,3]
df2.loc['d', 'two'] = 4

df2

,one,two
a,1,1
b,2,2
c,3,3
d,NaN,4


In [4]:
df2['two']

a    1
b    2
c    3
d    4
Name: two, dtype: object


#### xarray

Librería que buscar profundizar Pandas para poder trabajar con estructuras de datos más complejas. En particular trabaja con estructuras similares al formato NetCDF. Basicamente busca complejizar la estructura de arreglos de Numpy con metadatos.

##### Ejemplo de uso:


In [5]:
import numpy as np
import xarray

my_array = np.zeros((3,3,3), "float32")

# Nobre de dimensiones y coordenadas

my_name = "my_xarray"

my_dims = ["bottom_top", "south_north", "west_east"]

my_coords = {"bottom_top" : [100., 200., 300.],
          "south_north": [40., 50., 60.],
          "west_east" : [-120., -110., -100.]
         }

my_attrs = {"info" : "This is my xarray"}

my_xarray = xarray.DataArray(my_array,
                           name=my_name,
                           dims=my_dims, 
                           coords=my_coords, 
                           attrs=my_attrs)

print (my_xarray)

<xarray.DataArray 'my_xarray' (bottom_top: 3, south_north: 3, west_east: 3)>
array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]], dtype=float32)
Coordinates:
  * south_north  (south_north) float64 40.0 50.0 60.0
  * west_east    (west_east) float64 -120.0 -110.0 -100.0
  * bottom_top   (bottom_top) float64 100.0 200.0 300.0
Attributes:
    info:     This is my xarray


### 2. Cómo leer una salida de WRF 

El archivo que devuelve el WRF tiene el formato NetCDF y contiene toda la información relativa a la simulación. Basicamente se destacan 3 secciones a tener en cuenta: 

- La primera son datos globales propios del archivo, como puede ser la fecha de inicio, la versión utilizada de WRF, la resolución, la proyección usada, etc.

- La segunda son las dimensiones, tanto de la grilla, de la resolución vertical, y del tiempo

- Y la tercera son las variables.

Hay varias alternativas de programas para analizar un NetCDF (como por ejemplo ncdump, ncl_filedump, netcdf4-python, PyNIO, xarray, etc), algunos en Python y otros directamente por terminal.



#### netcdf4-python

Vamos a utilizar **netcdf4-python**, para poder verlo directamente desde la consola de Python. 


##### Datos globales

Una vez cargada la salida del WRF, se pueden listar todos los datos globales con * .\__dict\__ * , o también accediendo a cada variable particularmente con * .NOMBRE_VARIABLE *

In [6]:
from netCDF4 import Dataset

nc_file = Dataset("wrfout_d01_2017-11-29_06:00:00")

nc_file.START_DATE

u'2017-11-29_06:00:00'

In [7]:
nc_file.__dict__

OrderedDict([(u'TITLE', u' OUTPUT FROM WRF V3.9 MODEL'),
             (u'START_DATE', u'2017-11-29_06:00:00'),
             (u'SIMULATION_START_DATE', u'2017-11-29_06:00:00'),
             (u'WEST-EAST_GRID_DIMENSION', 270),
             (u'SOUTH-NORTH_GRID_DIMENSION', 270),
             (u'BOTTOM-TOP_GRID_DIMENSION', 35),
             (u'DX', 4000.0),
             (u'DY', 4000.0),
             (u'SKEBS_ON', 0),
             (u'SPEC_BDY_FINAL_MU', 1),
             (u'USE_Q_DIABATIC', 0),
             (u'GRIDTYPE', u'C'),
             (u'DIFF_OPT', 1),
             (u'KM_OPT', 4),
             (u'DAMP_OPT', 0),
             (u'DAMPCOEF', 0.2),
             (u'KHDIF', 0.0),
             (u'KVDIF', 0.0),
             (u'MP_PHYSICS', 8),
             (u'RA_LW_PHYSICS', 1),
             (u'RA_SW_PHYSICS', 2),
             (u'SF_SFCLAY_PHYSICS', 2),
             (u'SF_SURFACE_PHYSICS', 2),
             (u'BL_PBL_PHYSICS', 2),
             (u'CU_PHYSICS', 0),
             (u'SF_LAKE_PHYSICS',

##### Dimensiones

La grilla que utiliza WRF se compone de la siguiente manera:

![..](wrf_stagger.png)

- Las variables se conputan en los * mass points * que se ubican en el centro de la celda
- La componente u del viento se calcula en el limite derecho e izquierdo de la celda. En esta dirección se guarda un punto más.
- La componente v del viento se calcula en el limite superior e inferior de la celda. En esta dirección se guarda un punto más.
- Las esquinas de la celda son los * staggered points * , tiene un punto más en cada dirección. 


[link a wrf grid structure del sitio de mmm](http://www2.mmm.ucar.edu/rt/amps/information/configuration/wrf_grid_structure.html)

In [8]:
nc_file.dimensions

OrderedDict([(u'Time',
              <type 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'Time', size = 34),
             (u'DateStrLen',
              <type 'netCDF4._netCDF4.Dimension'>: name = 'DateStrLen', size = 19),
             (u'west_east',
              <type 'netCDF4._netCDF4.Dimension'>: name = 'west_east', size = 269),
             (u'south_north',
              <type 'netCDF4._netCDF4.Dimension'>: name = 'south_north', size = 269),
             (u'bottom_top',
              <type 'netCDF4._netCDF4.Dimension'>: name = 'bottom_top', size = 34),
             (u'bottom_top_stag',
              <type 'netCDF4._netCDF4.Dimension'>: name = 'bottom_top_stag', size = 35),
             (u'soil_layers_stag',
              <type 'netCDF4._netCDF4.Dimension'>: name = 'soil_layers_stag', size = 4),
             (u'west_east_stag',
              <type 'netCDF4._netCDF4.Dimension'>: name = 'west_east_stag', size = 270),
             (u'south_north_stag',
              <type 'netCDF4.

##### Variables

Esta estructura es la más compleja. Se pueden listar todas las variables y para cada una se almacena la siguiente estructura:

- atributos (datos generales de la variable, como unidad, coordenadas, descripción, etc)
- dimensiones (toma alguna de las dimensiones vistas antes)
- valores (los valores que toma para cada dimension)


In [9]:
# Para listar todos los nombres de variables posibles
nc_file.variables.keys()

[u'Times',
 u'XLAT',
 u'XLONG',
 u'LU_INDEX',
 u'ZNU',
 u'ZNW',
 u'ZS',
 u'DZS',
 u'VAR_SSO',
 u'U',
 u'V',
 u'W',
 u'PH',
 u'PHB',
 u'T',
 u'HFX_FORCE',
 u'LH_FORCE',
 u'TSK_FORCE',
 u'HFX_FORCE_TEND',
 u'LH_FORCE_TEND',
 u'TSK_FORCE_TEND',
 u'MU',
 u'MUB',
 u'NEST_POS',
 u'P',
 u'PB',
 u'FNM',
 u'FNP',
 u'RDNW',
 u'RDN',
 u'DNW',
 u'DN',
 u'CFN',
 u'CFN1',
 u'THIS_IS_AN_IDEAL_RUN',
 u'P_HYD',
 u'Q2',
 u'T2',
 u'TH2',
 u'PSFC',
 u'U10',
 u'V10',
 u'RDX',
 u'RDY',
 u'RESM',
 u'ZETATOP',
 u'CF1',
 u'CF2',
 u'CF3',
 u'ITIMESTEP',
 u'XTIME',
 u'QVAPOR',
 u'QCLOUD',
 u'QRAIN',
 u'QICE',
 u'QSNOW',
 u'QGRAUP',
 u'QNICE',
 u'QNRAIN',
 u'SHDMAX',
 u'SHDMIN',
 u'SNOALB',
 u'TSLB',
 u'SMOIS',
 u'SH2O',
 u'SMCREL',
 u'SEAICE',
 u'XICEM',
 u'SFROFF',
 u'UDROFF',
 u'IVGTYP',
 u'ISLTYP',
 u'VEGFRA',
 u'GRDFLX',
 u'ACGRDFLX',
 u'ACSNOM',
 u'SNOW',
 u'SNOWH',
 u'CANWAT',
 u'SSTSK',
 u'COSZEN',
 u'LAI',
 u'VAR',
 u'TKE_PBL',
 u'EL_PBL',
 u'MAPFAC_M',
 u'MAPFAC_U',
 u'MAPFAC_V',
 u'MAPFAC_MX',
 u'MAPFA

In [10]:
# Para listar los atributos de una variable
nc_file.variables['T'].__dict__

OrderedDict([(u'FieldType', 104),
             (u'MemoryOrder', u'XYZ'),
             (u'description',
              u'perturbation potential temperature (theta-t0)'),
             (u'units', u'K'),
             (u'stagger', u''),
             (u'coordinates', u'XLONG XLAT XTIME')])

In [11]:
# Para listar las dimensiones de una variable
nc_file.variables['T'].dimensions

(u'Time', u'bottom_top', u'south_north', u'west_east')

In [12]:
# Para ver el arreglo con los valores
nc_file.variables['T2'][:]

array([[[ 295.94186401,  296.04278564,  296.13766479, ...,  288.14498901,
          288.18127441,  288.23291016],
        [ 295.50531006,  295.65380859,  295.80123901, ...,  288.07849121,
          288.16699219,  288.25408936],
        [ 294.97113037,  295.17776489,  295.39007568, ...,  287.98153687,
          288.13549805,  288.2677002 ],
        ..., 
        [ 279.79547119,  280.1965332 ,  280.74349976, ...,  294.00469971,
          293.92666626,  293.85369873],
        [ 279.60070801,  280.01660156,  280.59326172, ...,  293.92098999,
          293.86279297,  293.80841064],
        [ 279.42370605,  279.84521484,  280.44824219, ...,  293.8387146 ,
          293.79699707,  293.75778198]],

       [[ 294.60928345,  294.5291748 ,  294.57723999, ...,  288.54177856,
          288.57104492,  288.60342407],
        [ 293.61376953,  293.57034302,  293.78771973, ...,  288.51425171,
          288.57388306,  288.62915039],
        [ 292.4954834 ,  292.54769897,  293.07275391, ...,  288.48181152

Por ejemplo para las componentes del viento, que toma valores en los bordes de las grillas, podemos ver que toma otras dimensiones

In [13]:
# Para listar los atributos de una variable
nc_file.variables['U'].__dict__

OrderedDict([(u'FieldType', 104),
             (u'MemoryOrder', u'XYZ'),
             (u'description', u'x-wind component'),
             (u'units', u'm s-1'),
             (u'stagger', u'X'),
             (u'coordinates', u'XLONG_U XLAT_U XTIME')])

In [14]:
# Para listar las dimensiones de una variable
nc_file.variables['U'].dimensions

(u'Time', u'bottom_top', u'south_north', u'west_east_stag')

### 3. wrf-python

**wrf-python** es un paquete desarrollado en Python por la NCAR (National Center for Atmospheric Research) que tiene una colección de cálculos diagnosticos y rutinas para interpolar (a niveles horizontales, * cross section *, a nivel de * superficie * ). Además provee funciones auxiliares para facilitar los gráficos con **cartopy** , ** basemap ** y ** PyNGL **

A pesar de que toda la interfaz esté en Python, utiliza rutinas definidas en Fortran para trabajar más eficientemente.

Algunas funciones útiles:
- getvar: extrae una variable o genera una variable diagnóstico nueva.
- ll_to_xy, xy_to_ll : funciones para convertir un punto de latitud/longitud a (x,y) de la grilla
- interplevel: interpolación lineal de una variable a un plano horizontal de un nivel vertival específico.
- vertcross: interpola una variable en 3D a una sección transversal
- vinterp: Interpola una variable a una nueva superficie

#### 3.1 getvar

Esta función se puede usar para extraer variables que ya estan en el NetCDF o generar variables nuevas. También se puede concatenar una variable a través de varios archivos NetCDF.

Por defecto extrae la variable para el tiempo 0, en caso de querer la variable para todos los tiempos del archivo hay que agregar timeidx=wrf.ALL_TIMES

En caso de tener la librería xarray cargada, la función devuelve la variable cargada con el tipo de DataArray de xarray.

Para ver la tabla de todas las posibles variables diagnositcos que se pueden generar: [link](http://wrf-python.readthedocs.io/en/latest/user_api/generated/wrf.getvar.html)

Ejemplo:


In [20]:
import wrf 

# Extrae la temperatura a 2 metros del nivel del suelo
t2 = wrf.getvar(nc_file, "T2", timeidx=wrf.ALL_TIMES)

t2

<xarray.DataArray 'T2' (Time: 34, south_north: 269, west_east: 269)>
array([[[ 295.941864,  296.042786, ...,  288.181274,  288.23291 ],
        [ 295.50531 ,  295.653809, ...,  288.166992,  288.254089],
        ..., 
        [ 279.600708,  280.016602, ...,  293.862793,  293.808411],
        [ 279.423706,  279.845215, ...,  293.796997,  293.757782]],

       [[ 294.609283,  294.529175, ...,  288.571045,  288.603424],
        [ 293.61377 ,  293.570343, ...,  288.573883,  288.62915 ],
        ..., 
        [ 277.444061,  275.336945, ...,  293.140015,  293.240814],
        [ 277.815918,  275.998383, ...,  293.170288,  293.131256]],

       ..., 
       [[ 297.552399,  296.956818, ...,  289.761719,  289.690094],
        [ 296.644623,  296.254669, ...,  289.823669,  289.742889],
        ..., 
        [ 281.649475,  278.085632, ...,  303.910156,  304.235291],
        [ 281.959015,  277.781189, ...,  304.113861,  304.06839 ]],

       [[ 298.957123,  298.302429, ...,  289.817413,  289.738281],

In [22]:
# Altura en km, al desactivar la opción msl (Median Sea Level) se calcula AGL (Above Ground Level)
z = wrf.getvar(nc_file, 'z', timeidx=wrf.ALL_TIMES, msl=False, units='km')

# Reflectividad simulada
dbz = wrf.getvar(nc_file, 'dbz', timeidx=wrf.ALL_TIMES)

dbz

<xarray.DataArray u'dbz' (Time: 34, bottom_top: 34, south_north: 269, west_east: 269)>
array([[[[-30., ..., -30.],
         ..., 
         [-30., ..., -30.]],

        ..., 
        [[-30., ..., -30.],
         ..., 
         [-30., ..., -30.]]],


       ..., 
       [[[-30., ..., -30.],
         ..., 
         [-30., ..., -30.]],

        ..., 
        [[-30., ..., -30.],
         ..., 
         [-30., ..., -30.]]]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 -68.9103 -68.8641 -68.818 ...
    XLAT     (south_north, west_east) float32 -37.2171 -37.2199 -37.2227 ...
    XTIME    (Time) float32 0.0 60.0 120.0 180.0 240.0 300.0 360.0 420.0 ...
  * Time     (Time) datetime64[ns] 2017-11-29T06:00:00 2017-11-29T07:00:00 ...
Dimensions without coordinates: bottom_top, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  radar reflectivity
    units:        dBZ
    stagger:      
    coordinates:  XLONG XLAT XTIME
    